In [4]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
from census import Census
from pprint import pprint
import gmaps

# Census API
from config import Census_API
c = Census(Census_API, year=2015)

In [5]:
population_by_language_dict = {"English":"B16001_002E","Spanish":"B16001_003E",
                              "French":"B16001_006E","Italian":"B16001_012E",
                              "Portuguese":"B16001_015E","German":"B16001_018E",
                              "Greek":"B16001_030E","Russian":"B16001_033E","Polish":"B16001_036E",
                              "Chinese":"B16001_066E","Japanese":"B16001_069E","Korean":"B16001_072E",
                              "Thai":"B16001_081E","Vietnamese":"B16001_087E","Arabic":"B16001_108E"}

language_name_list = list(population_by_language_dict.keys())
language_aliaese_list = list(population_by_language_dict.values())

In [6]:
# Get City ZipCode Data
city_zipcode_df = pd.read_csv("zip_code_database.csv")
city_zipcode_df.head()

,zip,type,decommissioned,primary_city,acceptable_cities,unacceptable_cities,state,county,timezone,area_codes,world_region,country,latitude,longitude,irs_estimated_population_2015
0,501,UNIQUE,0,Holtsville,NaN,I R S Service Center,NY,Suffolk County,America/New_York,631,NaN,US,40.81,-73.04,562
1,544,UNIQUE,0,Holtsville,NaN,Irs Service Center,NY,Suffolk County,America/New_York,631,NaN,US,40.81,-73.04,0
2,601,STANDARD,0,Adjuntas,NaN,"Colinas Del Gigante, Jard De Adjuntas, Urb San...",PR,Adjuntas Municipio,America/Puerto_Rico,"787,939",NaN,US,18.16,-66.72,0
3,602,STANDARD,0,Aguada,NaN,"Alts De Aguada, Bo Guaniquilla, Comunidad Las ...",PR,Aguada Municipio,America/Puerto_Rico,"787,939",NaN,US,18.38,-67.18,0
4,603,STANDARD,0,Aguadilla,Ramey,"Bda Caban, Bda Esteves, Bo Borinquen, Bo Ceiba...",PR,Aguadilla Municipio,America/Puerto_Rico,787,NaN,US,18.43,-67.15,0


In [7]:
# Extract zip code data for Chicago Only
column_output = ["zip","primary_city","state","latitude","longitude"]
city_zipcode_filter_df = pd.DataFrame(columns=column_output)

city_zipcode_filter_df = city_zipcode_df.loc[(city_zipcode_df["primary_city"]=="New York")&(city_zipcode_df["state"]=="NY"),column_output]

zipcode_list = list(pd.to_numeric(city_zipcode_filter_df["zip"]))

In [8]:
# Run Census Search to retrieve data on all states
# Note the addition of "B23025_005E" for unemployment count
census_data = c.acs5.get(language_aliaese_list+["B01003_001E"], {'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={**{"zip code tabulation area": "Zipcode","B01003_001E": "Population"},**dict(zip(language_aliaese_list,language_name_list))})


census_pd["Zipcode"] = pd.to_numeric(census_pd["Zipcode"])

census_Chicago_pd = census_pd.loc[census_pd["Zipcode"].isin(zipcode_list)]

census_Chicago_pd.head()

,Population,English,Spanish,French,Italian,Portuguese,German,Greek,Russian,Polish,Chinese,Japanese,Korean,Thai,Vietnamese,Arabic,Zipcode
2558,23537.0,13676.0,2398.0,543.0,188.0,157.0,139.0,79.0,170.0,50.0,1772.0,202.0,765.0,55.0,40.0,172.0,10001
2559,80736.0,28311.0,16666.0,765.0,108.0,23.0,134.0,57.0,472.0,114.0,28341.0,338.0,197.0,12.0,160.0,144.0,10002
2560,57112.0,42457.0,3203.0,1269.0,385.0,367.0,292.0,121.0,272.0,68.0,2672.0,618.0,1048.0,0.0,58.0,116.0,10003
2561,3221.0,2504.0,56.0,4.0,0.0,0.0,20.0,0.0,5.0,0.0,165.0,16.0,77.0,0.0,0.0,0.0,10004
2562,8131.0,6050.0,322.0,18.0,29.0,19.0,38.0,39.0,80.0,0.0,283.0,149.0,257.0,23.0,0.0,100.0,10005


In [9]:
census_Chicago_pd.to_csv("Population_by_Language_New_York.csv",index=False)